In [ ]:
from imutils.object_detection import non_max_suppression
import matplotlib.pyplot as plt
import numpy as np
import argparse
import time
import cv2

In [ ]:

image = cv2.imread("training_images/a8d6142c-dd7e-4863-9e54-50a1909c44f1.png")
# image = cv2.imread("training_images/a8d6142c-dd7e-4863-9e54-50a1909c44f1.png")

# image height and width should be multiple of 32
imgWidth=320
imgHeight=320

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
(thresh, image) = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)
image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

orig = image.copy()
(H, W) = image.shape[:2]
(newW, newH) = (imgWidth, imgHeight)

rW = W / float(newW)
rH = H / float(newH)
image = cv2.resize(image, (newW, newH))

(H, W) = image.shape[:2]
plt.imshow(image)
plt.show()

In [ ]:
net = cv2.dnn.readNet("frozen_east_text_detection.pb")

In [ ]:
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
                             (123.68, 116.78, 103.94), swapRB=True, crop=False)

In [ ]:
outputLayers = []
outputLayers.append("feature_fusion/Conv_7/Sigmoid")
outputLayers.append("feature_fusion/concat_3")

In [904]:
net.setInput(blob)
output = net.forward(outputLayers)
scores = output[0]
geometry = output[1]

In [ ]:
(numRows, numCols) = scores.shape[2:4]
rects = []
confidences = []

for y in range(0, numRows):
    scoresData = scores[0, 0, y]
    xData0 = geometry[0, 0, y]
    xData1 = geometry[0, 1, y]
    xData2 = geometry[0, 2, y]
    xData3 = geometry[0, 3, y]
    anglesData = geometry[0, 4, y]

    for x in range(0, numCols):
        # if our score does not have sufficient probability, ignore it
        if scoresData[x] < 0.5:
            continue

        # compute the offset factor as our resulting feature maps will
        # be 4x smaller than the input image
        (offsetX, offsetY) = (x * 4.0, y * 4.0)

        # extract the rotation angle for the prediction and then
        # compute the sin and cosine
        angle = anglesData[x]
        cos = np.cos(angle)
        sin = np.sin(angle)

        # use the geometry volume to derive the width and height of
        # the bounding box
        h = xData0[x] + xData2[x]
        w = xData1[x] + xData3[x]

        # compute both the starting and ending (x, y)-coordinates for
        # the text prediction bounding box
        endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
        endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
        startX = int(endX - w)
        startY = int(endY - h)

        # add the bounding box coordinates and probability score to
        # our respective lists
        rects.append((startX, startY, endX, endY))
        confidences.append(scoresData[x])

In [ ]:
boxes = non_max_suppression(np.array(rects), probs=confidences)

In [ ]:
height = 135
width = 410

In [ ]:
# boxes = boxes[boxes[:, 3].argsort()]
# diffs = [abs(boxes[i][3] - boxes[i - 1][3]) for i in range(1, len(boxes))]
# separation_locations = [i[0] for i in enumerate(diffs) if i[1] > 30]
# lines = [boxes[:separation_locations[0] + 1]] + [boxes[separation_locations[i - 1] + 1 : separation_locations[i] + 1] for i in range(1, len(separation_locations))] + [boxes[separation_locations[-1] + 1:]]
# lines = [line[line[:, 0].argsort()] for line in lines]

In [ ]:
# loop over the bounding boxes
lines = [1]
for line_boxes in lines:
    for (startX, startY, endX, endY) in boxes:
        # scale the bounding box  based on the respective
        # ratioscoordinates
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)

        # draw the bounding box on the image
        # cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)

        roi = orig[startY:endY, startX:endX]
        roi = cv2.resize(roi, (width, height))
        plt.imshow(roi)
        plt.show()


In [ ]:
plt.imshow(orig)
plt.show()